# Chapter 3
# The Homology Module
## Example

In [1]:
(ql:quickload "kenzo")

("kenzo")

In [2]:
(cat:cat-init)


---done---

NIL

In [3]:
(cat:def diabolo (cat:build-chcm
                  :cmpr #'cat:s-cmpr
                  :basis #'(lambda (dmn)
                                 (case dmn
                                     (0 '(s0 s1 s2 s3 s4 s5))
                                     (1 '(s01 s02 s12 s23 s34 s35 s45))
                                     (2 '(s345))
                                     (otherwise nil)))
                  :bsgn 's0
                  :intr-dffr #'(lambda (dmn gnr)
                                       (unless (<= 0 dmn 2)
                                           (error "Incorrect dimension for diabolo-dp."))
                                       (case dmn
                                           (0 (cat:cmbn -1))  ; Note the null combination of degree -1
                                           (1 (case gnr
                                                  (s01 (cat:cmbn 0 -1 's0 1 's1))
                                                  (s02 (cat:cmbn 0 -1 's0 1 's2))
                                                  (s12 (cat:cmbn 0 -1 's1 1 's2))
                                                  (s23 (cat:cmbn 0 -1 's2 1 's3))
                                                  (s34 (cat:cmbn 0 -1 's3 1 's4))
                                                  (s35 (cat:cmbn 0 -1 's3 1 's5))
                                                  (s45 (cat:cmbn 0 -1 's4 1 's5))))
                                           (2 (case gnr
                                                  (s345 (cat:cmbn 1 1 's34 -1 's35 1 's45))))
                                           (otherwise (error "Bad generator for complex diabolo"))))
                  :strt :GNRT
                  :orgn '(diabolo-for-example)))

[K1 Chain-Complex]

In [4]:
(cat:chcm-homology diabolo 0)


Computing boundary-matrix in dimension 0.
Rank of the source-module : 6.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 1/6 (dimension 0) :
S0 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 2/6 (dimension 0) :
S1 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 3/6 (dimension 0) :
S2 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 4/6 (dimension 0) :
S3 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 5/6 (dimension 0) :
S4 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 6/6 (dimension 0) :
S5 
End of computing.


Computing boundary-matrix in dimension 1.
Rank of the source-module : 7.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 1/7 (dimension 1) :
S01 
End of computing.


;

NIL

In [5]:
(cat:chcm-homology diabolo 1)


Computing boundary-matrix in dimension 1.
Rank of the source-module : 7.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 1/7 (dimension 1) :
S01 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 2/7 (dimension 1) :
S02 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 3/7 (dimension 1) :
S12 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 4/7 (dimension 1) :
S23 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 5/7 (dimension 1) :
S34 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 6/7 (dimension 1) :
S35 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 7/7 (dimension 1) :
S45 
End of computing.


Computing boundary-matrix in dimension 2.
Rank of the source-module : 

NIL

In [6]:
(cat:chcm-homology diabolo 2)


Computing boundary-matrix in dimension 2.
Rank of the source-module : 1.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 1/1 (dimension 2) :
S345 
End of computing.


Computing boundary-matrix in dimension 3.
Rank of the source-module : 0.




Homology in dimension 2 :



---done---

NIL

In [7]:
(cat:def duncehat-basis #'(lambda (dmn)
                                  (case dmn
                                      (0 '((0) (1) (2) (3) (4) (5) (6) (7)))
                                      (1 '((0 1) (0 2) (0 3)
                                           (0 4) (0 5) (0 6)
                                           (0 7) (1 2) (1 3)
                                           (1 4) (1 5) (1 6)
                                           (1 7) (2 3) (2 4)
                                           (2 6) (2 7) (3 4)
                                           (3 5) (4 5) (4 6)
                                           (5 6) (5 7) (6 7)))
                                      (2 '((0 1 5) (0 1 6) (0 1 7)
                                           (0 2 3) (0 2 4) (0 2 6)
                                           (0 3 4) (0 5 7) (1 2 3)
                                           (1 2 4) (1 2 7) (1 3 5)
                                           (1 4 6) (2 6 7) (3 4 5)
                                           (4 5 6) (5 6 7)))
                                      (otherwise nil))))

#<Anonymous Function #x302001DB640F>

In [8]:
(cat:def duncehat-df #'(lambda (dmn gnr)
                               (case dmn
                                   (0 (cat:cmbn -1))
                                   (1 (cat:cmbn 0 -1 (list (first gnr)) 1 (rest gnr)))
                                   (2 (cat:cmbn 1 1 (list (first gnr) (second gnr))
                                                 -1 (list (first gnr) (third gnr))
                                                  1 (rest gnr)))
                                   (otherwise nil))))

#<Anonymous Function #x302001DCAE4F>

In [9]:
(cat:def duncehat (cat:build-chcm :cmpr #'cat:l-cmpr
                                  :basis duncehat-basis
                                  :bsgn '(0)
                                  :intr-dffr duncehat-df
                                  :strt :gnrt
                                  :orgn '(dunce hat)))

[K3 Chain-Complex]

In [10]:
(cat:chcm-homology duncehat 0)


Computing boundary-matrix in dimension 0.
Rank of the source-module : 8.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 1/8 (dimension 0) :
(0) 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 2/8 (dimension 0) :
(1) 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 3/8 (dimension 0) :
(2) 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 4/8 (dimension 0) :
(3) 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 5/8 (dimension 0) :
(4) 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 6/8 (dimension 0) :
(5) 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 7/8 (dimension 0) :
(6) 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the gen

NIL

In [11]:
(cat:chcm-homology duncehat 1)


Computing boundary-matrix in dimension 1.
Rank of the source-module : 24.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 1/24 (dimension 1) :
(0 1) 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 2/24 (dimension 1) :
(0 2) 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 3/24 (dimension 1) :
(0 3) 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 4/24 (dimension 1) :
(0 4) 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 5/24 (dimension 1) :
(0 5) 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 6/24 (dimension 1) :
(0 6) 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 7/24 (dimension 1) :
(0 7) 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing t

NIL

In [12]:
(cat:chcm-homology duncehat 2)


Computing boundary-matrix in dimension 2.
Rank of the source-module : 17.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 1/17 (dimension 2) :
(0 1 5) 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 2/17 (dimension 2) :
(0 1 6) 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 3/17 (dimension 2) :
(0 1 7) 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 4/17 (dimension 2) :
(0 2 3) 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 5/17 (dimension 2) :
(0 2 4) 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 6/17 (dimension 2) :
(0 2 6) 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 7/17 (dimension 2) :
(0 3 4) 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14

NIL

In [13]:
(cat:def mz (cat:chcm-mat duncehat 1))


Computing boundary-matrix in dimension 1.
Rank of the source-module : 24.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 1/24 (dimension 1) :
(0 1) 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 2/24 (dimension 1) :
(0 2) 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 3/24 (dimension 1) :
(0 3) 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 4/24 (dimension 1) :
(0 4) 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 5/24 (dimension 1) :
(0 5) 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 6/24 (dimension 1) :
(0 6) 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 7/24 (dimension 1) :
(0 7) 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing t


========== MATRIX 8 lines + 24 columns =====
L1=[C1=-1][C2=-1][C3=-1][C4=-1][C5=-1][C6=-1][C7=-1]
L2=[C1=1][C8=-1][C9=-1][C10=-1][C11=-1][C12=-1][C13=-1]
L3=[C2=1][C8=1][C14=-1][C15=-1][C16=-1][C17=-1]
L4=[C3=1][C9=1][C14=1][C18=-1][C19=-1]
L5=[C4=1][C10=1][C15=1][C18=1][C20=-1][C21=-1]
L6=[C5=1][C11=1][C19=1][C20=1][C22=-1][C23=-1]
L7=[C6=1][C12=1][C16=1][C21=1][C22=1][C24=-1]
L8=[C7=1][C13=1][C17=1][C23=1][C24=1]
========== END-MATRIX

In [14]:
(cat:def nb (cat:chcm-mat duncehat 2))


Computing boundary-matrix in dimension 2.
Rank of the source-module : 17.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 1/17 (dimension 2) :
(0 1 5) 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 2/17 (dimension 2) :
(0 1 6) 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 3/17 (dimension 2) :
(0 1 7) 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 4/17 (dimension 2) :
(0 2 3) 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 5/17 (dimension 2) :
(0 2 4) 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 6/17 (dimension 2) :
(0 2 6) 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 7/17 (dimension 2) :
(0 3 4) 
End of computing.


;; Clock -> 2015-08-21, 19h 37m 14


========== MATRIX 24 lines + 17 columns =====
L1=[C1=1][C2=1][C3=1]
L2=[C4=1][C5=1][C6=1]
L3=[C4=-1][C7=1]
L4=[C5=-1][C7=-1]
L5=[C1=-1][C8=1]
L6=[C2=-1][C6=-1]
L7=[C3=-1][C8=-1]
L8=[C9=1][C10=1][C11=1]
L9=[C9=-1][C12=1]
L10=[C10=-1][C13=1]
L11=[C1=1][C12=-1]
L12=[C2=1][C13=-1]
L13=[C3=1][C11=-1]
L14=[C4=1][C9=1]
L15=[C5=1][C10=1]
L16=[C6=1][C14=1]
L17=[C11=1][C14=-1]
L18=[C7=1][C15=1]
L19=[C12=1][C15=-1]
L20=[C15=1][C16=1]
L21=[C13=1][C16=-1]
L22=[C16=1][C17=1]
L23=[C8=1][C17=-1]
L24=[C14=1][C17=1]
========== END-MATRIX

In [15]:
(cat:homologie mz nb)

NIL

## 3.2 The general method for computing homology
### Example

In [16]:
(cat:cat-init)


---done---

NIL

In [17]:
(cat:def s2 (cat:sphere 2))

[K1 Simplicial-Set]

In [18]:
(describe s2)

[K1 Simplicial-Set]
Class: #<STANDARD-CLASS CAT::SIMPLICIAL-SET>
Wrapper: #<CCL::CLASS-WRAPPER CAT::SIMPLICIAL-SET #x302001B2688D>
Instance slots
CAT:CMPR: #<Compiled-function CAT::SPHERE-CMPR #x302001C19EFF>
CAT:BASIS: #<CCL:COMPILED-LEXICAL-CLOSURE (:INTERNAL
                                           CAT::RSLT
                                           CAT::SPHERE-BASIS) #x302001E3230F>
CAT:BSGN: *
CAT:DFFR: [K2 Morphism (degree -1): K1 -> K1]
CAT::GRMD: [K1 Simplicial-Set]
CAT:EFHM: #<Unbound>
CAT:IDNM: 1
CAT:ORGN: (CAT:SPHERE 2)
CAT:CPRD: [K5 Morphism (degree 0): K1 -> K3]
CAT:FACE: #<CCL:COMPILED-LEXICAL-CLOSURE (:INTERNAL
                                          CAT::RSLT
                                          CAT:SPHERE-FACE) #x302001E322AF>


NIL

In [19]:
(cat:homology s2 1)


Computing boundary-matrix in dimension 1.
Rank of the source-module : 0.


Computing boundary-matrix in dimension 2.
Rank of the source-module : 1.


;; Clock -> 2015-08-21, 19h 37m 14s.
Computing the boundary of the generator 1/1 (dimension 2) :
S2 
End of computing.




Homology in dimension 1 :



---done---

;; Clock -> 2015-08-21, 19h 37m 14s.



NIL

In [20]:
(describe s2)

[K1 Simplicial-Set]
Class: #<STANDARD-CLASS CAT::SIMPLICIAL-SET>
Wrapper: #<CCL::CLASS-WRAPPER CAT::SIMPLICIAL-SET #x302001B2688D>
Instance slots
CAT:CMPR: #<Compiled-function CAT::SPHERE-CMPR #x302001C19EFF>
CAT:BASIS: #<CCL:COMPILED-LEXICAL-CLOSURE (:INTERNAL
                                           CAT::RSLT
                                           CAT::SPHERE-BASIS) #x302001E3230F>
CAT:BSGN: *
CAT:DFFR: [K2 Morphism (degree -1): K1 -> K1]
CAT::GRMD: [K1 Simplicial-Set]
CAT:EFHM: [K9 Homotopy-Equivalence K1 <= K1 => K1]
CAT:IDNM: 1
CAT:ORGN: (CAT:SPHERE 2)
CAT:CPRD: [K5 Morphism (degree 0): K1 -> K3]
CAT:FACE: #<CCL:COMPILED-LEXICAL-CLOSURE (:INTERNAL
                                          CAT::RSLT
                                          CAT:SPHERE-FACE) #x302001E322AF>


NIL

In [21]:
(cat:orgn (cat:hmeq 9))

(CAT:TRIVIAL-HMEQ [K1 Simplicial-Set])

In [22]:
(cat:def os2 (cat:loop-space s2))

[K10 Simplicial-Group]

In [23]:
(describe os2)

[K10 Simplicial-Group]
Class: #<STANDARD-CLASS CAT:SIMPLICIAL-GROUP>
Wrapper: #<CCL::CLASS-WRAPPER CAT:SIMPLICIAL-GROUP #x302001B4C3ED>
Instance slots
CAT:CMPR: #<CCL:COMPILED-LEXICAL-CLOSURE (:INTERNAL
                                          CAT::RSLT
                                          CAT:LOOP-SPACE-CMPR) #x302001ED76CF>
CAT:BASIS: :LOCALLY-EFFECTIVE
CAT:BSGN: <<Loop>>
CAT:DFFR: [K11 Morphism (degree -1): K10 -> K10]
CAT::GRMD: [K10 Simplicial-Group]
CAT:EFHM: #<Unbound>
CAT:IDNM: 10
CAT:ORGN: (CAT:LOOP-SPACE [K1 Simplicial-Set])
CAT:APRD: #<Unbound>
CAT:CPRD: [K14 Morphism (degree 0): K10 -> K12]
CAT:FACE: #<CCL:COMPILED-LEXICAL-CLOSURE (:INTERNAL
                                          CAT::RSLT
                                          CAT:LOOP-SPACE-FACE) #x302001ED767F>
CAT:KFLL: #<Unbound>
CAT:GRML: [K20 Simplicial-Morphism K15 -> K10]
CAT:GRIN: [K21 Simplicial-Morphism K10 -> K10]


NIL

In [24]:
(cat:orgn os2)

(CAT:LOOP-SPACE [K1 Simplicial-Set])

In [25]:
(cat:efhm os2)

[K118 Homotopy-Equivalence K10 <= K108 => K25]

In [26]:
(describe os2)

[K10 Simplicial-Group]
Class: #<STANDARD-CLASS CAT:SIMPLICIAL-GROUP>
Wrapper: #<CCL::CLASS-WRAPPER CAT:SIMPLICIAL-GROUP #x302001B4C3ED>
Instance slots
CAT:CMPR: #<CCL:COMPILED-LEXICAL-CLOSURE (:INTERNAL
                                          CAT::RSLT
                                          CAT:LOOP-SPACE-CMPR) #x302001D0426F>
CAT:BASIS: :LOCALLY-EFFECTIVE
CAT:BSGN: <<Loop>>
CAT:DFFR: [K11 Morphism (degree -1): K10 -> K10]
CAT::GRMD: [K10 Simplicial-Group]
CAT:EFHM: [K118 Homotopy-Equivalence K10 <= K108 => K25]
CAT:IDNM: 10
CAT:ORGN: (CAT:LOOP-SPACE [K1 Simplicial-Set])
CAT:APRD: #<Unbound>
CAT:CPRD: [K14 Morphism (degree 0): K10 -> K12]
CAT:FACE: #<CCL:COMPILED-LEXICAL-CLOSURE (:INTERNAL
                                          CAT::RSLT
                                          CAT:LOOP-SPACE-FACE) #x302001D0421F>
CAT:KFLL: #<Unbound>
CAT:GRML: [K20 Simplicial-Morphism K15 -> K10]
CAT:GRIN: [K21 Simplicial-Morphism K10 -> K10]


NIL

In [27]:
(cat:def s4 (cat:sphere 4))

[K119 Simplicial-Set]

In [28]:
(describe s4)

[K119 Simplicial-Set]
Class: #<STANDARD-CLASS CAT::SIMPLICIAL-SET>
Wrapper: #<CCL::CLASS-WRAPPER CAT::SIMPLICIAL-SET #x302001B2688D>
Instance slots
CAT:CMPR: #<Compiled-function CAT::SPHERE-CMPR #x302001C19EFF>
CAT:BASIS: #<CCL:COMPILED-LEXICAL-CLOSURE (:INTERNAL
                                           CAT::RSLT
                                           CAT::SPHERE-BASIS) #x302001DB506F>
CAT:BSGN: *
CAT:DFFR: [K120 Morphism (degree -1): K119 -> K119]
CAT::GRMD: [K119 Simplicial-Set]
CAT:EFHM: #<Unbound>
CAT:IDNM: 119
CAT:ORGN: (CAT:SPHERE 4)
CAT:CPRD: [K123 Morphism (degree 0): K119 -> K121]
CAT:FACE: #<CCL:COMPILED-LEXICAL-CLOSURE (:INTERNAL
                                          CAT::RSLT
                                          CAT:SPHERE-FACE) #x302001DB500F>


NIL

In [29]:
(cat:def ooos4 (cat:loop-space (cat:loop-space (cat:loop-space s4))))

[K148 Simplicial-Group]

In [30]:
(cat:orgn ooos4)

(CAT:LOOP-SPACE [K136 Simplicial-Group])

In [31]:
(cat:orgn (second (cat:orgn ooos4)))

(CAT:LOOP-SPACE [K124 Simplicial-Group])

In [32]:
(cat:orgn (second (cat:orgn (second (cat:orgn ooos4)))))

(CAT:LOOP-SPACE [K119 Simplicial-Set])

In [33]:
(cat:orgn (second (cat:orgn (second (cat:orgn (second (cat:orgn ooos4)))))))

(CAT:SPHERE 4)

In [34]:
(describe (cat:smgr 136))

[K136 Simplicial-Group]
Class: #<STANDARD-CLASS CAT:SIMPLICIAL-GROUP>
Wrapper: #<CCL::CLASS-WRAPPER CAT:SIMPLICIAL-GROUP #x302001B4C3ED>
Instance slots
CAT:CMPR: #<CCL:COMPILED-LEXICAL-CLOSURE (:INTERNAL
                                          CAT::RSLT
                                          CAT:LOOP-SPACE-CMPR) #x302001DF573F>
CAT:BASIS: :LOCALLY-EFFECTIVE
CAT:BSGN: <<Loop>>
CAT:DFFR: [K137 Morphism (degree -1): K136 -> K136]
CAT::GRMD: [K136 Simplicial-Group]
CAT:EFHM: #<Unbound>
CAT:IDNM: 136
CAT:ORGN: (CAT:LOOP-SPACE [K124 Simplicial-Group])
CAT:APRD: #<Unbound>
CAT:CPRD: [K140 Morphism (degree 0): K136 -> K138]
CAT:FACE: #<CCL:COMPILED-LEXICAL-CLOSURE (:INTERNAL
                                          CAT::RSLT
                                          CAT:LOOP-SPACE-FACE) #x302001DF56EF>
CAT:KFLL: #<Unbound>
CAT:GRML: [K146 Simplicial-Morphism K141 -> K136]
CAT:GRIN: [K147 Simplicial-Morphism K136 -> K136]


NIL

In [35]:
(describe (cat:smgr 124))

[K124 Simplicial-Group]
Class: #<STANDARD-CLASS CAT:SIMPLICIAL-GROUP>
Wrapper: #<CCL::CLASS-WRAPPER CAT:SIMPLICIAL-GROUP #x302001B4C3ED>
Instance slots
CAT:CMPR: #<CCL:COMPILED-LEXICAL-CLOSURE (:INTERNAL
                                          CAT::RSLT
                                          CAT:LOOP-SPACE-CMPR) #x302001DFA6EF>
CAT:BASIS: :LOCALLY-EFFECTIVE
CAT:BSGN: <<Loop>>
CAT:DFFR: [K125 Morphism (degree -1): K124 -> K124]
CAT::GRMD: [K124 Simplicial-Group]
CAT:EFHM: #<Unbound>
CAT:IDNM: 124
CAT:ORGN: (CAT:LOOP-SPACE [K119 Simplicial-Set])
CAT:APRD: #<Unbound>
CAT:CPRD: [K128 Morphism (degree 0): K124 -> K126]
CAT:FACE: #<CCL:COMPILED-LEXICAL-CLOSURE (:INTERNAL
                                          CAT::RSLT
                                          CAT:LOOP-SPACE-FACE) #x302001DFA69F>
CAT:KFLL: #<Unbound>
CAT:GRML: [K134 Simplicial-Morphism K129 -> K124]
CAT:GRIN: [K135 Simplicial-Morphism K124 -> K124]


NIL

In [36]:
(describe (cat:smst 119))

[K119 Simplicial-Set]
Class: #<STANDARD-CLASS CAT::SIMPLICIAL-SET>
Wrapper: #<CCL::CLASS-WRAPPER CAT::SIMPLICIAL-SET #x302001B2688D>
Instance slots
CAT:CMPR: #<Compiled-function CAT::SPHERE-CMPR #x302001C19EFF>
CAT:BASIS: #<CCL:COMPILED-LEXICAL-CLOSURE (:INTERNAL
                                           CAT::RSLT
                                           CAT::SPHERE-BASIS) #x302001DB506F>
CAT:BSGN: *
CAT:DFFR: [K120 Morphism (degree -1): K119 -> K119]
CAT::GRMD: [K119 Simplicial-Set]
CAT:EFHM: #<Unbound>
CAT:IDNM: 119
CAT:ORGN: (CAT:SPHERE 4)
CAT:CPRD: [K123 Morphism (degree 0): K119 -> K121]
CAT:FACE: #<CCL:COMPILED-LEXICAL-CLOSURE (:INTERNAL
                                          CAT::RSLT
                                          CAT:SPHERE-FACE) #x302001DB500F>


NIL

In [37]:
(cat:efhm ooos4)

[K522 Homotopy-Equivalence K148 <= K512 => K508]

In [38]:
(describe (cat:smgr 124))

[K124 Simplicial-Group]
Class: #<STANDARD-CLASS CAT:SIMPLICIAL-GROUP>
Wrapper: #<CCL::CLASS-WRAPPER CAT:SIMPLICIAL-GROUP #x302001B4C3ED>
Instance slots
CAT:CMPR: #<CCL:COMPILED-LEXICAL-CLOSURE (:INTERNAL
                                          CAT::RSLT
                                          CAT:LOOP-SPACE-CMPR) #x302001D2179F>
CAT:BASIS: :LOCALLY-EFFECTIVE
CAT:BSGN: <<Loop>>
CAT:DFFR: [K125 Morphism (degree -1): K124 -> K124]
CAT::GRMD: [K124 Simplicial-Group]
CAT:EFHM: [K426 Homotopy-Equivalence K124 <= K416 => K329]
CAT:IDNM: 124
CAT:ORGN: (CAT:LOOP-SPACE [K119 Simplicial-Set])
CAT:APRD: #<Unbound>
CAT:CPRD: [K128 Morphism (degree 0): K124 -> K126]
CAT:FACE: #<CCL:COMPILED-LEXICAL-CLOSURE (:INTERNAL
                                          CAT::RSLT
                                          CAT:LOOP-SPACE-FACE) #x302001D2174F>
CAT:KFLL: #<Unbound>
CAT:GRML: [K134 Simplicial-Morphism K129 -> K124]
CAT:GRIN: [K135 Simplicial-Morphism K124 -> K124]


NIL

In [39]:
(describe (cat:smst 119))

[K119 Simplicial-Set]
Class: #<STANDARD-CLASS CAT::SIMPLICIAL-SET>
Wrapper: #<CCL::CLASS-WRAPPER CAT::SIMPLICIAL-SET #x302001B2688D>
Instance slots
CAT:CMPR: #<Compiled-function CAT::SPHERE-CMPR #x302001C19EFF>
CAT:BASIS: #<CCL:COMPILED-LEXICAL-CLOSURE (:INTERNAL
                                           CAT::RSLT
                                           CAT::SPHERE-BASIS) #x302001D1823F>
CAT:BSGN: *
CAT:DFFR: [K120 Morphism (degree -1): K119 -> K119]
CAT::GRMD: [K119 Simplicial-Set]
CAT:EFHM: [K412 Homotopy-Equivalence K119 <= K119 => K119]
CAT:IDNM: 119
CAT:ORGN: (CAT:SPHERE 4)
CAT:CPRD: [K123 Morphism (degree 0): K119 -> K121]
CAT:FACE: #<CCL:COMPILED-LEXICAL-CLOSURE (:INTERNAL
                                          CAT::RSLT
                                          CAT:SPHERE-FACE) #x302001D181DF>


NIL

In [40]:
(describe s4)

[K119 Simplicial-Set]
Class: #<STANDARD-CLASS CAT::SIMPLICIAL-SET>
Wrapper: #<CCL::CLASS-WRAPPER CAT::SIMPLICIAL-SET #x302001B2688D>
Instance slots
CAT:CMPR: #<Compiled-function CAT::SPHERE-CMPR #x302001C19EFF>
CAT:BASIS: #<CCL:COMPILED-LEXICAL-CLOSURE (:INTERNAL
                                           CAT::RSLT
                                           CAT::SPHERE-BASIS) #x302001D1823F>
CAT:BSGN: *
CAT:DFFR: [K120 Morphism (degree -1): K119 -> K119]
CAT::GRMD: [K119 Simplicial-Set]
CAT:EFHM: [K412 Homotopy-Equivalence K119 <= K119 => K119]
CAT:IDNM: 119
CAT:ORGN: (CAT:SPHERE 4)
CAT:CPRD: [K123 Morphism (degree 0): K119 -> K121]
CAT:FACE: #<CCL:COMPILED-LEXICAL-CLOSURE (:INTERNAL
                                          CAT::RSLT
                                          CAT:SPHERE-FACE) #x302001D181DF>


NIL